In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

churn = pd.read_csv('./data/churn.csv', sep=',', header=0)

churn.columns = [heading.lower() for heading in churn.columns.str.replace(' ', '_').str.replace("\'", "").str.strip('?')]

churn['churn01'] = np.where(churn['churn'] == 'True.', 1., 0.)
print(churn.head())

  state  account_length  area_code     phone intl_plan vmail_plan  \
0    KS             128        415  382-4657        no        yes   
1    OH             107        415  371-7191        no        yes   
2    NJ             137        415  358-1921        no         no   
3    OH              84        408  375-9999       yes         no   
4    OK              75        415  330-6626       yes         no   

   vmail_message  day_mins  day_calls  day_charge  ...  eve_charge  \
0             25     265.1        110       45.07  ...       16.78   
1             26     161.6        123       27.47  ...       16.62   
2              0     243.4        114       41.38  ...       10.30   
3              0     299.4         71       50.90  ...        5.26   
4              0     166.7        113       28.34  ...       12.61   

   night_mins  night_calls  night_charge  intl_mins  intl_calls  intl_charge  \
0       244.7           91         11.01       10.0           3         2.70   
1   

In [3]:
print(churn.groupby(['churn'])
      [['day_charge', 'eve_charge', 'night_charge', 'intl_charge', 'account_length', 'custserv_calls']]
      .agg(['count', 'mean', 'std']))

       day_charge                       eve_charge                       \
            count       mean        std      count       mean       std   
churn                                                                     
False.       2850  29.780421   8.530835       2850  16.918909  4.274863   
True.         483  35.175921  11.729710        483  18.054969  4.396762   

       night_charge                     intl_charge                      \
              count      mean       std       count      mean       std   
churn                                                                     
False.         2850  9.006074  2.299768        2850  2.743404  0.751784   
True.           483  9.235528  2.121081         483  2.889545  0.754152   

       account_length                       custserv_calls                      
                count        mean       std          count      mean       std  
churn                                                                           
False

In [4]:
print(churn.groupby(['churn']).agg({'day_charge': ['mean', 'std'],
                                   'eve_charge': ['mean', 'std'],
                                   'night_charge': ['mean', 'std'],
                                   'intl_charge': ['mean', 'std'],
                                   'account_length': ['count', 'min', 'max'],
                                   'custserv_calls': ['count', 'min', 'max']}))

       day_charge            eve_charge           night_charge            \
             mean        std       mean       std         mean       std   
churn                                                                      
False.  29.780421   8.530835  16.918909  4.274863     9.006074  2.299768   
True.   35.175921  11.729710  18.054969  4.396762     9.235528  2.121081   

       intl_charge           account_length          custserv_calls          
              mean       std          count min  max          count min max  
churn                                                                        
False.    2.743404  0.751784           2850   1  243           2850   0   8  
True.     2.889545  0.754152            483   1  225            483   0   9  


In [5]:
churn['total_charges'] = churn['day_charge'] + churn['eve_charge'] + churn['night_charge'] + churn['intl_charge']

factor_cut = pd.cut(churn.total_charges, 5, precision=2)

def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 
            'count': group.count(), 'mean': group.mean(),
            'std': group.std()}

grouped = churn.custserv_calls.groupby(factor_cut)
print(grouped.apply(get_stats).unstack())

                min  max   count      mean       std
total_charges                                       
(22.86, 37.57]  0.0  5.0    70.0  1.528571  1.348337
(37.57, 52.22]  0.0  7.0   742.0  1.564690  1.305234
(52.22, 66.86]  0.0  9.0  1726.0  1.581692  1.326646
(66.86, 81.51]  0.0  9.0   735.0  1.523810  1.295209
(81.51, 96.15]  0.0  5.0    60.0  1.516667  1.359108


In [6]:
factor_qcut = pd.qcut(churn.account_length, [0., 0.25, 0.5, 0.75, 1.])
grouped = churn.custserv_calls.groupby(factor_qcut)
print(grouped.apply(get_stats).unstack())

                min  max  count      mean       std
account_length                                     
(0.999, 74.0]   0.0  9.0  857.0  1.506418  1.251268
(74.0, 101.0]   0.0  7.0  847.0  1.604486  1.359888
(101.0, 127.0]  0.0  8.0  803.0  1.652553  1.358479
(127.0, 243.0]  0.0  9.0  826.0  1.491525  1.286970


In [7]:
intl_dummies = pd.get_dummies(churn['intl_plan'], prefix='intl_plan')
vmail_dummies = pd.get_dummies(churn['vmail_plan'], prefix='vmail_plan')
churn_with_dummies = churn[['churn']].join([intl_dummies, vmail_dummies])
print(churn_with_dummies.head())

    churn  intl_plan_no  intl_plan_yes  vmail_plan_no  vmail_plan_yes
0  False.             1              0              0               1
1  False.             1              0              0               1
2  False.             1              0              1               0
3  False.             0              1              1               0
4  False.             0              1              1               0


In [8]:
qcut_names = ['1st_quartile', '2nd_quartile', '3rd_quartile', '4rd_quartile']
total_charges_quartiles = pd.qcut(churn.total_charges, 4, labels=qcut_names)
dummies = pd.get_dummies(total_charges_quartiles, prefix='total_charges')
churn_with_dummies = churn.join(dummies)
print(churn_with_dummies.head())

  state  account_length  area_code     phone intl_plan vmail_plan  \
0    KS             128        415  382-4657        no        yes   
1    OH             107        415  371-7191        no        yes   
2    NJ             137        415  358-1921        no         no   
3    OH              84        408  375-9999       yes         no   
4    OK              75        415  330-6626       yes         no   

   vmail_message  day_mins  day_calls  day_charge  ...  intl_calls  \
0             25     265.1        110       45.07  ...           3   
1             26     161.6        123       27.47  ...           3   
2              0     243.4        114       41.38  ...           5   
3              0     299.4         71       50.90  ...           7   
4              0     166.7        113       28.34  ...           3   

   intl_charge  custserv_calls   churn  churn01  total_charges  \
0         2.70               1  False.      0.0          75.56   
1         3.70               1  

In [9]:
print(churn.pivot_table(['total_charges'], index=['churn', 'custserv_calls']))

                       total_charges
churn  custserv_calls               
False. 0                   58.429752
       1                   58.164391
       2                   57.534241
       3                   58.797195
       4                   64.318000
       5                   66.303077
       6                   62.150000
       7                   64.677500
       8                   64.670000
True.  0                   69.601087
       1                   70.723443
       2                   69.399080
       3                   68.931136
       4                   55.374474
       5                   52.845500
       6                   49.714286
       7                   50.578000
       8                   52.730000
       9                   70.390000


In [10]:
print(churn.pivot_table(['total_charges'], index=['churn'], columns=['custserv_calls']))

               total_charges                                              \
custserv_calls             0          1          2          3          4   
churn                                                                      
False.             58.429752  58.164391  57.534241  58.797195  64.318000   
True.              69.601087  70.723443  69.399080  68.931136  55.374474   

                                                             
custserv_calls          5          6        7      8      9  
churn                                                        
False.          66.303077  62.150000  64.6775  64.67    NaN  
True.           52.845500  49.714286  50.5780  52.73  70.39  


In [11]:
print(churn.pivot_table(['total_charges'], index=['custserv_calls'], columns=['churn'], aggfunc='mean', fill_value='NaN', margins=True))

               total_charges                      
churn                 False.      True.        All
custserv_calls                                    
0                    58.4298  69.601087  59.904304
1                    58.1644  70.723443  59.461770
2                    57.5342  69.399080  58.894242
3                    58.7972  68.931136  59.836573
4                     64.318  55.374474  60.223373
5                    66.3031  52.845500  58.146970
6                      62.15  49.714286  54.236364
7                    64.6775  50.578000  56.844444
8                      64.67  52.730000  58.700000
9                        NaN  70.390000  70.390000
All                  58.4488  65.355963  59.449754


## 1 로지스틱 회귀분석

In [12]:
dependent_variable = churn['churn01']
independent_variables = churn[['account_length', 'custserv_calls', 'total_charges']]
independent_variables_with_constant = sm.add_constant(independent_variables, prepend=True)
logit_model = sm.Logit(dependent_variable, independent_variables_with_constant).fit()

print(logit_model.summary())
print('\nQuantities you can extract from the result:\n%s' % dir(logit_model))
print('\nCoefficients:\n%s' % logit_model.params)
print('\nCoefficient Std Errors:\n%s' % logit_model.bse)

Optimization terminated successfully.
         Current function value: 0.363480
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                churn01   No. Observations:                 3333
Model:                          Logit   Df Residuals:                     3329
Method:                           MLE   Df Model:                            3
Date:                Tue, 27 Aug 2019   Pseudo R-squ.:                  0.1216
Time:                        23:45:29   Log-Likelihood:                -1211.5
converged:                       True   LL-Null:                       -1379.1
Covariance Type:            nonrobust   LLR p-value:                 2.234e-72
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -7.2205      0.394    -18.309      0.000      -7.993      -6.448
account_length   

/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
